In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import PIL
print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
import re
import random

In [ ]:
# @title Define image loading and visualization functions  { display-mode: "form" }

def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), crop=False, preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  image_path = image_url
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(image_path),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]

  if not image_size==0:
    img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  if crop == True:
    img = crop_center(img)
  return img


def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

In [ ]:
hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

In [ ]:
def stylise_frames(hub_module, input_frame_path, style_frame_path, content_img_size, style_img_size):
    content_image = load_image(input_frame_path, content_img_size)
    style_image = load_image(style_frame_path, style_img_size, crop=True)
    style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    stylised_image = outputs[0]
    return stylised_image

In [ ]:
def save_tensor_image(tensor, output_path):
    si_np = tensor.numpy()
    si_np = np.squeeze(si_np, axis=0)
    si_np = (si_np * 255).astype(np.uint8)
    im1 = PIL.Image.fromarray(si_np, 'RGB')
    im1.save(output_path)

In [ ]:
content_image_url = "/Users/iman/github/avignon/projects/graphite/graphite_original/0002.png"
style_image_url = "/Users/iman/github/avignon/projects/graphite/temp/0001.png"

content_img_size = (1920, 1080)
style_img_size = (int(1920/4), int(1080/4))  # Recommended to keep it at 256.

content_image = load_image(content_image_url, 0)
print(content_image.shape)
style_image = load_image(style_image_url, style_img_size, crop=False)
style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
# show_n([content_image, style_image], ['Content image', 'Style image'])

outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
stylized_image = outputs[0]

show_n([content_image, style_image, stylized_image], titles=['Original content image', 'Style image', 'Stylized image'])
save_tensor_image(stylized_image, "/Users/iman/github/avignon/other/text_out2.jpg")

In [ ]:
content_image.shape

In [ ]:
folder_path = '/Users/iman/github/avi/test2/keyframes'

file_numbers = []

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    filename = os.path.basename(filename)
    if 'testimony' in filename:
        file_numbers.append(filename.split('testimony')[1].split('.')[0])

In [ ]:
out_imgs = []

In [ ]:
for n in file_numbers:
    out_img = stylise_frames(hub_module, input_frame_path=f"/Users/iman/github/avi/test2/keyframes/testimony{n}.jpg", style_frame_path=f"/Users/iman/github/avi/test2/keyframes/testimony{n}.jpg",content_img_size=(1280, 720),style_img_size=(1280, 720))
# show_n([out_img], titles=["Result"])
    out_imgs.append(out_img)

In [ ]:
style_folder_path = '/Users/iman/github/avi/test2/keyframes_s/'

for i, img in enumerate(out_imgs):
    n = file_numbers[i]
    output_frame_path = style_folder_path+"testimony"+n+".jpg"
    save_tensor_image(img, output_frame_path)

In [ ]:
import os
import glob
import shutil 
import PIL, numpy

In [ ]:
input_folder_path = '/Users/iman/frames_s/'  # Replace with the actual folder path
style_folder_path = '/Users/iman/frames_r/'  # Replace with the actual folder path
output_folder_path = '/Users/iman/frames_o/'

In [ ]:
if os.path.exists(output_folder_path):
    shutil.rmtree(output_folder_path)
os.mkdir(output_folder_path)

jpeg_files = glob.glob(os.path.join(input_folder_path, '*.jpg'))
jpeg_files.sort()

In [ ]:
content_img_size=(1280, 720)
style_img_size=(256*3, 256)

In [ ]:
for file_path in jpeg_files:
    # Perform operations on each JPEG file
    print(file_path) 
    input_frame_path = file_path
    style_frame_path = os.path.join(os.path.dirname(style_folder_path), os.path.basename(file_path))
    output_frame_path = os.path.join(os.path.dirname(output_folder_path), os.path.basename(file_path))
    output_frame = stylise_frames(hub_module, input_frame_path=input_frame_path, style_frame_path=style_frame_path,content_img_size=content_img_size,style_img_size=style_img_size)
    # show_n([output_frame], titles=["Result"])
    save_tensor_image(output_frame, output_frame_path)

In [ ]:
stylise_frames(hub_module, input_frame_path=input_frame_path, style_frame_path=style_frame_path,content_img_size=content_img_size,style_img_size=style_img_size)
    save_tensor_image(output_frame, output_frame_path)

In [ ]:
import cv2

# Read two consecutive frames from the video
frame1 = cv2.imread('/Users/iman/github/avignon/projects/graphite/output/0000.png')
frame2 = cv2.imread('/Users/iman/github/avignon/projects/graphite/output/0199.png')

# Specify the number of new frames to create
num_frames = 10

# Convert frames to grayscale
# frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)


# Calculate the interpolation step size
interpolation_step = 1 / (num_frames + 1)

# Create and save the new frames
for i in range(1, num_frames + 1):
    # Calculate the interpolation factor
    interpolation_factor = i * interpolation_step

    # Perform the interpolation
    interpolated_frame = cv2.addWeighted(frame1, interpolation_factor, frame2, 1 - interpolation_factor, 0)

    # Save the interpolated frame
    cv2.imwrite(f'interpolated_frame_{i}.jpg', interpolated_frame)

    # Display the interpolated frame
    show_image(interpolated_frame)

In [ ]:
def show_image(image, fig_size=(10,20)):
    image = np.squeeze(image)
    plt.figure(figsize=fig_size)
    plt.imshow(image)
    plt.axis('off')
    return plt.show()

In [ ]:
import glob

In [ ]:

frames_directory = '/Users/iman/github/avignon/projects/graphite/output'
output_video_path = '/Users/iman/output_video.mp4'
command_buffer = {}

input_files = glob.glob(os.path.join(frames_directory, '*.png'))
input_files.sort()
interpolation_factor = 0.5
frame_rate = 270
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video = cv2.VideoWriter(output_video_path, fourcc, 30, (1080, 1920))  # Assuming frame size is 640x480
do_blur = 0
# kernel_sizes = [num for num in range(2, 1000) if num % 2 != 0]
kernel_sizes = [3,5,7]
skip_percentage = 5

for i, frame_path in enumerate(input_files):
    random_number = random.randint(0,100)
    frame_duration = int(1000 / frame_rate)
    current_frame = cv2.imread(frame_path) 
    next_frame =  cv2.imread(input_files[min(len(input_files)-1,i+1)]) 
    interpolated_frame = cv2.addWeighted(current_frame, interpolation_factor, next_frame, 1 - interpolation_factor, 0)
    
    if random_number < skip_percentage:
        do_blur = do_blur + 10

    if do_blur:
        kernel_size = (kernel_sizes[min(len(kernel_sizes)-1,do_blur)], kernel_sizes[min(len(kernel_sizes)-1,do_blur)])  # Adjust the kernel size based on the desired blur effect
        interpolated_frame = cv2.GaussianBlur(interpolated_frame, kernel_size, 0)
        do_blur = do_blur - 1

    for _ in range(frame_duration):
        output_video.write(interpolated_frame)
    
    print(do_blur)
    
# Release the video writer
output_video.release()

In [ ]:
1000/300

In [ ]:
    interpolated_frame = cv2.GaussianBlur(interpolated_frame, kernel_size, 0)
    if random_number < skip_percentage:
        frames_skip = random.randint(max(0,i-20), min((i+20),len(input_files)-1))
        i = frames_skip
        current_frame = interpolated_frame
        next_frame = cv2.imread(input_files[i])
        interpolated_frame = cv2.addWeighted(current_frame, interpolation_factor, next_frame, 1 - interpolation_factor, 0)

In [ ]:
do_blur